In [1]:
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
from numpy.random import rand
from numpy.random import randint
from numpy.random import randn
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import UpSampling2D
import keras
from keras import layers
import keras.backend as K
from keras.layers import LeakyReLU

In [2]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.analysis.structure_matcher import StructureMatcher
atom_dict1=['na','H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg','Al','Si','P','S','Cl','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Xe','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr','Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn','Nh','Fl','Mc','Lv','Ts','Og']
#removing nobels 
atom_dict=['na','H','Li','Be','B','C','N','O','F','Na','Mg','Al','Si','P','S','Cl','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr','Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn','Nh','Fl','Mc','Lv','Ts','Og']

In [ ]:
#Load functions for latent points and generator

In [14]:
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [16]:
def generate_fake_samples(n_samples):
    X=generate_latent_points(512*80,n_samples)
    X=X.reshape((n_samples,160,256))/(np.max(np.abs(X)))
    X=(X+1)/2
    y = np.zeros((n_samples,1))
    return X,y

In [20]:
def define_generator():
    #Generator
    visible = keras.layers.Input(shape=(128,))
    n_nodes = 128 * 16 * 10
    dense = keras.layers.Dense(n_nodes)(visible)
    reshape1 = keras.layers.Reshape((10, 16, 128))(dense)
    upsample1 = keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(reshape1)
    LeakyRelu1 = keras.layers.LeakyReLU(alpha=0.2)(upsample1)
    upsample2 = keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(LeakyRelu1)
    LeakyRelu2 = keras.layers.LeakyReLU(alpha=0.2)(upsample2)
    upsample3 = keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(LeakyRelu2)
    LeakyRelu3 = keras.layers.LeakyReLU(alpha=0.2)(upsample3)
    upsample4 = keras.layers.Conv2DTranspose(64, (2,2), strides=(1,4), padding='same')(LeakyRelu3)
    LeakyRelu4 = keras.layers.LeakyReLU(alpha=0.2)(upsample4)
    conv_layer1 = Conv2D(1, (2,2), activation='sigmoid', padding='same')(LeakyRelu4)
    #output = crstldisc(conv_layer1)
    generator = keras.Model(visible, conv_layer1)
    generator.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
    #generator.summary()
    return generator

In [21]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(128,n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X[:,:,:,0], y

In [ ]:
#Load atom + lattice decoder, load CHGNet and CGCNN for use as discriminator

import atomate2
from atomate2.forcefields.flows.phonons import PhononMaker
from pymatgen.core.structure import Structure
from jobflow import run_locally
from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
from pymatgen.phonon.dos import PhononDos
from pymatgen.phonon.plotter import PhononBSPlotter, PhononDosPlotter
from jobflow import SETTINGS
from numpy import genfromtxt
import subprocess
import os
from jobflow import run_locally

from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
from pymatgen.phonon.dos import PhononDos
from pymatgen.phonon.plotter import PhononBSPlotter, PhononDosPlotter
from jobflow import SETTINGS


import keras
from keras import layers
from keras.layers import LeakyReLU
import keras.backend as K
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

input_img = keras.Input(shape=(64,64,64, 1))
x = layers.BatchNormalization()(input_img)
x = layers.Conv3D(16, 3, strides = (4,4,4),  padding='same')(input_img)
x = LeakyReLU(alpha=0.2)(x)
x = layers.Conv3D(8, 3, strides = (4,4,4),activation='tanh', padding='same')(x)
encoded = layers.Flatten()(x)

x = layers.Dense(512, activation=None)(encoded)
x = layers.Reshape((4,4,4,8))(x)
x = layers.Conv3DTranspose(4, 8, strides=(4,4,4), padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
x = layers.Conv3DTranspose(8, 8, strides=(4,4,4),  padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
decoded = layers.Conv3D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
#autoencoder.compile(optimizer='adam', loss=K.binary_crossentropy, metrics=['mae'])
#autoencoder.summary()
apos1 = autoencoder

# Restore the weights
apos1.load_weights('AtomWeights.weights.h5')

import keras.backend as K

def custom_loss(y_true, y_pred):
    a=K.sum(K.abs(K.log(K.abs(y_true[:,31,0,0,0]))-K.log(K.abs(y_pred[:,31,0,0,0])))*2)
    b=K.sum(K.abs(K.log(K.abs(y_true[:,0,31,0,0]))-K.log(K.abs(y_pred[:,0,31,0,0])))*2)
    c=K.sum(K.abs(K.log(K.abs(y_true[:,0,0,31,0]))-K.log(K.abs(y_pred[:,0,0,31,0])))*2)
    ab=K.sum(K.abs(K.log(K.abs(y_true[:,31,31,0,0]))-K.log(K.abs(y_pred[:,31,31,0,0])))*2)
    bc=K.sum(K.abs(K.log(K.abs(y_true[:,0,31,31,0]))-K.log(K.abs(y_pred[:,0,31,31,0])))*2)
    ac=K.sum(K.abs(K.log(K.abs(y_true[:,31,0,31,0]))-K.log(K.abs(y_pred[:,31,0,31,0])))*2)
    loss1=a+b+c+ab+bc+ac
    loss2 = K.mean(K.abs(y_true - y_pred))

    err=loss1+10*loss2

    return err
import keras
from keras import layers
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

input_img = keras.Input(shape=(32, 32, 32, 1))

x = layers.Conv3D(16, 3, strides = (2,2,2), activation='relu', padding='same')(input_img)
x = layers.Conv3D(16, 3,strides = (2,2,2), activation='relu', padding='same')(x)
x = layers.Conv3D(8, 3, strides = (2,2,2),activation='tanh', padding='same')(x)
encoded = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(encoded)
x = layers.Reshape((4,4,4,8))(x)
x = layers.Conv3DTranspose(4, 8, strides=(2,2,2), activation='relu', padding='same')(x)
x = layers.Conv3DTranspose(8, 8, strides=(2,2,2), activation='relu', padding='same')(x)
x = layers.Conv3DTranspose(16, 16, strides=(2,2,2), activation='relu', padding='same')(x)
decoded = layers.Conv3D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
#autoencoder.compile(optimizer='adam', loss=custom_loss, metrics=[custom_loss,'mae'])
#autoencoder.summary()
apos=autoencoder

# Restore the weights
apos.load_weights('LatticeWeights.weights.h5')

import tensorflow as tf
from tensorflow import keras as keras
import keras
from keras import layers
#apos1 = keras.models.load_model('Atom_Encoder.keras')
full_input = keras.Input(shape=(64,64,64,1), name='full_input')
new_encoded_1 = apos1.layers[1](full_input)
new_encoded_2 = apos1.layers[2](new_encoded_1)
new_encoded_3 = apos1.layers[3](new_encoded_2)
new_encoded_4 = apos1.layers[4](new_encoded_3)

atomencoder = keras.Model(full_input, new_encoded_4)

#Now Decode 
encoded_input = keras.Input(shape=(512,), name='encoded_input')
new_decoded_0 = apos1.layers[5](encoded_input)
new_decoded_1 = apos1.layers[6](new_decoded_0)
new_decoded_2 = apos1.layers[7](new_decoded_1)
new_decoded_3 = apos1.layers[8](new_decoded_2)
new_decoded_4 = apos1.layers[9](new_decoded_3)
new_decoded_5 = apos1.layers[10](new_decoded_4)
new_decoded_6 = apos1.layers[11](new_decoded_5)
#new_decoded_7 = apos.layers[12](new_decoded_6)

atomdecoder = keras.Model(encoded_input, new_decoded_6)

#apos = keras.models.load_model('Lattice_Encoder.keras', compile=False)
full_input = keras.Input(shape=(32,32,32,1), name='full_input')
new_encoded_1 = apos.layers[1](full_input)
new_encoded_2 = apos.layers[2](new_encoded_1)
new_encoded_3 = apos.layers[3](new_encoded_2)
new_encoded_4 = apos.layers[4](new_encoded_3)

latencoder = keras.Model(full_input, new_encoded_4)

#Now Decode 
encoded_input = keras.Input(shape=(512,), name='encoded_input')
new_decoded_0 = apos.layers[5](encoded_input)
new_decoded_1 = apos.layers[6](new_decoded_0)
new_decoded_2 = apos.layers[7](new_decoded_1)
new_decoded_3 = apos.layers[8](new_decoded_2)
new_decoded_4 = apos.layers[9](new_decoded_3)
new_decoded_5 = apos.layers[10](new_decoded_4)

latdecoder = keras.Model(encoded_input, new_decoded_5)


def format_output(out):
    n=np.shape(out)[0]
    out=2*out-1
    atoms=out[:,1:,:]
    lats=out[:,:1,:]
    return lats, atoms

def elements(decoded,tol1):
    atm_lst=[]
    for j in range(1,79):

        prob=(np.max(decoded[j,:,:,:,0]))
        #print(prob)
        if prob>tol1:
            atm_lst.append(j)
    return atm_lst
#Make a list with atom locations
def atmloc(decoded,atmnum,tol):
    atomlocation=[]
    jj=int(atmnum)
    #for jj in [atmnum]:
    testa=decoded[jj:jj+1,:,:,:,:]
    #print(np.shape(test))
    a=testa[0,:,:,:,0]
    #a=img[2001]
    sorted_index_array = np.argsort((a.flatten()))
    sorted_array = (a.flatten())[sorted_index_array]
    rslt = sorted_array[-10000 : ]
    atom_coords=[]
    found=False
    j=1
    while found==False:
        if rslt[-j]>tol:
            crd= np.where(a==rslt[-j])
            ijk=[crd[0][0],crd[1][0],crd[2][0]]
            if j==1:
                atom_coords.append(ijk)
            #if num_atoms==1:
            #    found=True
            else:
                dist=[]
                for point in atom_coords:
                    
                    dist.append(np.linalg.norm(np.array(ijk)-np.array(point))*(12/64))
                    #Introduce Periodicity here
                if np.min(dist)>2:
                    atom_coords.append(ijk)
            #if (np.shape(np.array(atom_coords))[0])==int(num_atoms):
            #    found=True
        j=j+1
        if j==10000:
            atomlocation.append(np.array(atom_coords)*(12/64))
            found=True
    return atomlocation

def extract_lat(latdecoded):
    a=(-np.log(latdecoded[0,-1,0,0,0])*2) #a
    b=(-np.log(latdecoded[0,0,-1,0,0])*2) #b
    c=(-np.log(latdecoded[0,0,0,-1,0])*2)+15 #c
    g=((np.arccos((-np.log(latdecoded[0,-1,-1,0,0])*2-a-b)/(a*b)))*(180/np.pi))
    return a, b, c, g





import chgnet
from chgnet.model import CHGNet
from chgnet.model import StructOptimizer
chgnet = CHGNet.from_file('01-08-2025/epoch19_e410_f282_sNA_mNA.pth.tar')
relaxer = StructOptimizer()
# Perturb the structure
def PhononDisc(xf, i1, i2):
    size=np.shape(xf)[0]
    samples=xf
    Res1=np.zeros(size)
    Res2=np.zeros(size)
    lats, atoms = format_output(samples)
    tol=0.3
    tol1=0.3
    for n in range(size):
        #try:
        #print(n)
        decodedatoms=atomdecoder.predict(atoms[n])
        #print(np.max(decodedatoms))
        atm_lst = elements(decodedatoms,tol1)
        #print(atm_lst)
        atm_names=[]
        atm_crd=[]
        if np.shape(atm_lst)[0]<20 and np.shape(atm_lst)[0]>1:
            for k1 in atm_lst:
                atom_location= atmloc(decodedatoms,k1,tol)
                for k2 in range(np.shape(atom_location)[1]):
                    atm_names.append(atom_dict[k1])
                    atm_crd.append(atom_location[0][k2])
            if np.shape(atm_names)[0]<20:
                latdecoded=latdecoder.predict(lats[n])
                a1, b1, c1, g=extract_lat(latdecoded)
                #Now build structure in pymatgen
                coords = atm_crd
                names=atm_names
                lattice = Lattice.from_parameters(a=a1, b=b1, c=c1, alpha=90,
                                                  beta=90, gamma=g)
                struct = Structure(lattice, names, coords,coords_are_cartesian=True)
                dist=struct.distance_matrix
                atom_cut=np.min(dist+10*np.diag(np.ones(np.shape(dist)[0])),axis=0)
                check_d=np.shape(dist)[0]**2-np.count_nonzero(np.heaviside(np.abs(dist+2*np.diag(np.ones(np.shape(dist)[0])))-1,0))
                if check_d==0 and np.max(atom_cut)<6:
                    print(struct)
                    struct.perturb(0.1)
                    result = relaxer.relax(struct, verbose=False)
                    struct=result["final_structure"]
                    phonon_flow = PhononMaker(min_length=15.0, store_force_constants=False).make(structure=struct)
                    run_locally(phonon_flow, create_folders=True)
                    store = SETTINGS.JOB_STORE
                    store.connect()
                    result = store.query_one(
                        {"name": "generate_frequencies_eigenvectors"},
                        properties=[
                            "output.phonon_dos",
                            "output.phonon_bandstructure",
                        ],
                        load=True,
                        sort={"completed_at": -1} # to get the latest computation
                    )
                    ph_dos = PhononDos.from_dict(result['output']['phonon_dos']) # get pymatgen phonon dos object
                    #stable=[False,True][int(np.heaviside(1-np.sum(np.abs(np.heaviside(-ph_dos.frequencies,0)*ph_dos.densities)),0))]
                    stable=1-np.clip(np.sum(np.heaviside(-ph_dos.frequencies,0)*np.clip(ph_dos.densities,0,.05)),0,1)
                    print('Phonon Negative Densities: '+str(stable))
                    Res1[n]=stable
                    os.mkdir('Temp')
                    subprocess.call('scp atom_init.json Temp', shell=True)
                    subprocess.call('scp id_prop.csv Temp', shell=True)
                    struct.to(filename="./Temp/temp.cif")
                    subprocess.call('python predict.py model_best.pth.tar ../Temp', shell=True, cwd='./cgcnn-master')
                    gap_res = genfromtxt('cgcnn-master/test_results.csv', delimiter=',')
                    subprocess.call('rm -r Temp', shell=True)
                    subprocess.call('rm -rf job_*', shell=True)
                    print("Predicted band gap: "+str(gap_res[2]))
                    Res2[n]=np.clip((gap_res[2])+.5,0,1)
                    if gap_res[2]>0 and stable>0:
                        #Res[n]=1
                        name=str(struct.formula).replace(" ", "")
                        #struct.to(filename="./Final_CIFS/Untrained/Good_Ins_"+str(n)+"_"+str(i1)+"_"+str(i2)+"_"+str(name)+".cif")
                        
    return Res1, Res2





/opt/anaconda3/envs/tf2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-04 18:15:47.209608: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-02-04 18:15:47.209636: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-02-04 18:15:47.209643: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-02-04 18:15:47.209660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-04 18:15:47.209670: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps


In [25]:
#Overfit on results of phonon density of states + CGCCNN gap prediction to use as discriminator 
def define_mlp_stab_discriminator():
    input_img0 = keras.Input(shape=( 80, 512, 1))
    x = layers.Conv2D(4, 2, strides = (2,4),  padding='same')(input_img0)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 2,strides = (2,2),  padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 3, strides = (2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Flatten()(x)
    output= layers.Dense(1, activation='sigmoid')(x)
    mlp_disc_stab = keras.Model(input_img0, output)
    mlp_disc_stab.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #crstldisc.summary()
    return mlp_disc_stab
def define_mlp_gap_discriminator():
    input_img1 = keras.Input(shape=( 80, 512, 1))
    x = layers.Conv2D(4, 2, strides = (2,4),  padding='same')(input_img1)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 2,strides = (2,2),  padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(32, 3, strides = (2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = layers.Flatten()(x)
    output= layers.Dense(1, activation='sigmoid')(x)
    mlp_disc_gap = keras.Model(input_img1, output)
    mlp_disc_gap.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #crstldisc.summary()
    return mlp_disc_gap

In [26]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, n_samples, x_input):
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X[:,:,:,0], y
def define_gan_mlp(g_model, mlp_stability_disc, mlp_gap_disc):
    # make weights in the discriminator not trainable
    mlp_stability_disc.trainable = False
    mlp_gap_disc.trainable = False
    # connect them
    input_img = keras.Input(shape=( 128,))
    # add generator
    gen = g_model(input_img)
    # add disc
    stab = mlp_stability_disc(gen)
    gap = mlp_gap_disc(gen)
    # compile model
    opt = Adam(learning_rate=0.0001, beta_1=0.5)
    model = keras.Model(inputs=input_img, outputs=[stab,gap])
    model.compile(loss=['binary_crossentropy','binary_crossentropy'], optimizer=opt, metrics=['accuracy','accuracy'])
    return model

In [27]:
def train(g_model):
    half_batch = int(64)
    g2_loss=[]
    #g2_loss=np.load('MLP_loss.npy')
    #g2_loss=g2_loss.tolist()
    # manually enumerate epochs
    for i in range(150,200):
        X_gan=generate_latent_points(128,half_batch)
        ##########################################################
        # generate 'fake' examples
        g_model=keras.models.load_model('g_model_mlp.keras')
        X_fake, y_fake = generate_fake_samples(g_model, half_batch, X_gan)
        ##########################################################
        #Check for dynamic stability
        res1, res2 =PhononDisc(X_fake, i, 0)
        mlp_stability_disc=define_mlp_stab_discriminator()
        mlp_stability_disc=keras.models.load_model('mlp_stab.keras')
        mlp_gap_disc=define_mlp_gap_discriminator()
        mlp_gap_disc=keras.models.load_model('mlp_gap.keras')
        Y1=np.expand_dims(np.array(res1),1)
        mlp_stability_disc.fit(X_fake,Y1,batch_size=64,epochs=300,verbose=0)
        Y2=np.expand_dims(np.array(res2),1)
        mlp_gap_disc.fit(X_fake,Y2,batch_size=64,epochs=300,verbose=0)
        ##########################################################
        # create inverted labels for the fake samples
        y_gan = np.ones((half_batch, 1))
        #Train w/ MLP
        mlp_stability_disc.trainable = False
        mlp_gap_disc.trainable = False
        # connect them
        input_img = keras.Input(shape=( 128,))
        # add generator
        gen = g_model(input_img)
        # add disc
        stab = mlp_stability_disc(gen)
        gap = mlp_gap_disc(gen)
        # compile model
        opt = Adam(learning_rate=0.0001, beta_1=0.5)
        gan_model2 = keras.Model(inputs=input_img, outputs=[stab,gap])
        gan_model2.compile(loss=['binary_crossentropy','binary_crossentropy'], optimizer=opt, metrics=['accuracy','accuracy'])
        #gan_model2 = define_gan2(g_model,mlp_stability_disc, mlp_gap_disc)
        g_loss2= gan_model.train_on_batch(X_gan, [y_gan, y_gan])
        print("GAN Loss: "+str(g_loss2))
        g2_loss.append(g_loss2)
        np.save('MLP_loss.npy',np.array(g2_loss))
        print('Epoch '+str(i))
        g_model.save('g_model_mlp.keras')
        #save models for this epoch 
        g_model.save('g_model_mlp_Epoch_'+str(i)+'.keras')
    
        

In [ ]:
g_model = define_generator()
#g_model=keras.models.load_model('g_model_mlp_Epoch_50.keras')
g_model.save('g_model_mlp.keras')
mlp_stability_disc=define_mlp_stab_discriminator()
mlp_stability_disc.save('mlp_stab.keras')
mlp_gap_disc=define_mlp_gap_discriminator()
mlp_gap_disc.save('mlp_gap.keras')
train(g_model)
